In [ ]:
# Step 1: Import Libraries and Initialize Constants
import os
import shutil
import glob
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns

In [ ]:
# For Google Colab drive mounting
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Constants and Configurations
IMG_SIZE = (224, 224)
BATCH_SIZE = 64
LEARNING_RATE = 0.0005
EPOCHS = 30
ADVERSARIAL_EPOCHS = 10  # Number of epochs for teacher adversarial training
EPSILON = 0.1
TEST_RATIO = 0.3
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Step 2: Define Models
class SimpleStudentModel(nn.Module):
    def __init__(self):
        super(SimpleStudentModel, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        # For binary classification, output two logits
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 2)

    def forward(self, x):
        return self.resnet(x)

class EnsembleModel(nn.Module):
    def __init__(self):
        super(EnsembleModel, self).__init__()
        self.densenet = models.densenet121(pretrained=True)
        self.resnet = models.resnet50(pretrained=True)
        self.efficientnet = models.efficientnet_b0(pretrained=True)

        # Freeze feature extractor parameters
        for model in [self.densenet, self.resnet, self.efficientnet]:
            for param in model.parameters():
                param.requires_grad = False

        # Replace classifier layers to produce 512-d features
        self.densenet.classifier = nn.Linear(self.densenet.classifier.in_features, 512)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 512)
        # efficientnet.classifier is typically a Sequential; replace its last layer
        self.efficientnet.classifier = nn.Sequential(
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(self.efficientnet.classifier[1].in_features, 512)
        )

        # Combined classifier for binary classification
        self.classifier = nn.Linear(512 * 3, 2)

    def forward(self, x):
        x1 = self.densenet(x)
        x2 = self.resnet(x)
        x3 = self.efficientnet(x)
        x_cat = torch.cat([x1, x2, x3], dim=1)
        out = self.classifier(x_cat)
        return out

In [ ]:
# Step 3: Data Loading and Transformations
test_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

validation_dataset = datasets.ImageFolder('/content/drive/MyDrive/BreakHis_dataset_split/validation', transform=test_transforms)
test_dataset = datasets.ImageFolder('/content/drive/MyDrive/BreakHis_dataset_split/test', transform=test_transforms)

validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [ ]:
# Step 4: FGSM Attack Function
def fgsm_attack(model, data, target, epsilon):
    """Generate adversarial examples using the FGSM method."""
    data.requires_grad = True
    output = model(data)
    loss = nn.CrossEntropyLoss()(output, target)
    model.zero_grad()
    loss.backward()
    data_grad = data.grad.data
    perturbation = epsilon * data_grad.sign()
    perturbed_data = data + perturbation
    # Ensure pixel values remain in [0,1]
    perturbed_data = torch.clamp(perturbed_data, 0, 1)
    return perturbed_data

In [ ]:
# Step 5: PGD Attack
def pgd_attack(model, data, target, epsilon, alpha=0.01, iters=10):
    original = data.clone().detach()
    for _ in range(iters):
        data.requires_grad = True
        output = model(data)
        loss = nn.CrossEntropyLoss()(output, target)
        model.zero_grad()
        loss.backward()
        data = data + alpha * data.grad.data.sign()
        eta = torch.clamp(data - original, min=-epsilon, max=epsilon)
        data = torch.clamp(original + eta, 0, 1).detach_()
    return data

In [ ]:
# Step 6: Transfer Attack
def transfer_attack(attacker, victim, loader, epsilon, attack_fn):
    correct, total = 0, 0
    for data, target in loader:
        data, target = data.to(DEVICE), target.to(DEVICE)
        adv_data = attack_fn(attacker, data.clone(), target, epsilon)
        output = victim(adv_data)
        pred = output.argmax(dim=1)
        correct += (pred == target).sum().item()
        total += target.size(0)
    return 100. * correct / total

In [ ]:
# Step 7: Load Student Saved Models
student_model = torch.load('/content/drive/MyDrive/Submission/student_model_BreakHis_full_trainingV1.0.pth', map_location=DEVICE, weights_only=False)
student_model.eval()

SimpleStudentModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True,

In [ ]:
# Step 8: Load Teacher Saved Models
teacher_model = torch.load('/content/drive/MyDrive/Submission/teacher_model_BreakHis_full_trainingV1.0.pth', map_location=DEVICE, weights_only=False)
teacher_model.eval()

EnsembleModel(
  (densenet): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momen

In [ ]:
# Step 9: Load Student Distilled Saved Models
# Load the state dictionary
state_dict = torch.load('/content/drive/MyDrive/Submission/best_student_model_distilled.pth', map_location=DEVICE)

# Instantiate the model and load the state dictionary
student_model_distilled = SimpleStudentModel().to(DEVICE)
student_model_distilled.load_state_dict(state_dict)

student_model_distilled.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 235MB/s]


SimpleStudentModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True,

In [ ]:
# Step 10: Visualize Learned Ensemble Weights
with torch.no_grad():
    # Access the weights of the 'classifier' attribute
    weights = teacher_model.classifier.weight

    # Reshape weights to (num_classes, num_models, features_per_model)
    # Assuming 2 classes, 3 models, and 512 features per model
    reshaped_weights = weights.reshape(2, 3, 512)

    # Calculate average weight magnitude for each model
    avg_weights_per_model = reshaped_weights.abs().mean(dim=[0, 2])

    # Normalize to get relative importance
    normalized_weights = avg_weights_per_model / avg_weights_per_model.sum()

    print("\nLearned Ensemble Weights (Normalized):")
    print(f"DenseNet121:   {normalized_weights[0].item():.4f}")
    print(f"ResNet50:      {normalized_weights[1].item():.4f}")
    print(f"EfficientNetB0:{normalized_weights[2].item():.4f}")


Learned Ensemble Weights (Normalized):
DenseNet121:   0.3633
ResNet50:      0.3239
EfficientNetB0:0.3127


In [ ]:
# Step 11: Evaluate the Student Model Before and After FGSM Attack
def evaluate_model(model, loader, epsilon=0, attack="fgsm"):
    model.eval()
    correct, total = 0, 0
    for data, target in loader:
        data, target = data.to(DEVICE), target.to(DEVICE)
        if epsilon > 0:
            if attack == "fgsm":
                data = fgsm_attack(model, data.clone(), target, epsilon)
            elif attack == "pgd":
                data = pgd_attack(model, data.clone(), target, epsilon)
        output = model(data)
        pred = output.argmax(dim=1)
        correct += (pred == target).sum().item()
        total += target.size(0)
    return 100. * correct / total

In [ ]:
torch.cuda.empty_cache()

In [ ]:
print("\n=== FGSM Accuracy Comparison ===")
print("Epsilon | Teacher FGSM | Student FGSM | Distilled FGSM")

eps0 = 0
eps001 = 0.01
eps005 = 0.05
eps01 = 0.1
eps02 = 0.2
eps03 = 0.3

r0 = (
    evaluate_model(teacher_model, test_loader, eps0, attack="fgsm"),
    evaluate_model(student_model, test_loader, eps0, attack="fgsm"),
    evaluate_model(student_model_distilled, test_loader, eps0, attack="fgsm"),
)
print(f"{eps0:>7} | {r0[0]:>13.2f} | {r0[1]:>13.2f} | {r0[2]:>15.2f}")


=== FGSM Accuracy Comparison ===
Epsilon | Teacher FGSM | Student FGSM | Distilled FGSM
      0 |         72.74 |         99.69 |           78.88


In [ ]:
epsilons = [0, 0.01, 0.05, 0.1, 0.2, 0.3]

print("\n=== Accuracy Comparison (FGSM / PGD / Transfer FGSM) ===")
print("Epsilon | Teacher FGSM | Student FGSM | Distilled FGSM | Teacher PGD | Student PGD | Distilled PGD | Transfer FGSM (Student→Teacher) | Transfer FGSM (Teacher→Student) | Transfer FGSM (Teacher→Distilled)")

for eps in epsilons:
    # FGSM
    acc_fgsm_teacher = evaluate_model(teacher_model, test_loader, eps, attack="fgsm")
    acc_fgsm_student = evaluate_model(student_model, test_loader, eps, attack="fgsm")
    acc_fgsm_distilled = evaluate_model(student_model_distilled, test_loader, eps, attack="fgsm")

    # PGD
    acc_pgd_teacher = evaluate_model(teacher_model, test_loader, eps, attack="pgd")
    acc_pgd_student = evaluate_model(student_model, test_loader, eps, attack="pgd")
    acc_pgd_distilled = evaluate_model(student_model_distilled, test_loader, eps, attack="pgd")

    # Transfer attacks
    acc_transfer_s2t = transfer_attack(student_model, teacher_model, test_loader, eps, fgsm_attack)
    acc_transfer_t2s = transfer_attack(teacher_model, student_model, test_loader, eps, fgsm_attack)
    acc_transfer_t2d = transfer_attack(teacher_model, student_model_distilled, test_loader, eps, fgsm_attack)

    print(f"{eps:>7} | {acc_fgsm_teacher:>13.2f} | {acc_fgsm_student:>13.2f} | {acc_fgsm_distilled:>15.2f} |"
          f"{acc_pgd_teacher:>12.2f} | {acc_pgd_student:>12.2f} | {acc_pgd_distilled:>14.2f} |"
          f"{acc_transfer_s2t:>28.2f} | {acc_transfer_t2s:>30.2f} | {acc_transfer_t2d:>31.2f}")


=== Accuracy Comparison (FGSM / PGD / Transfer FGSM) ===
Epsilon | Teacher FGSM | Student FGSM | Distilled FGSM | Teacher PGD | Student PGD | Distilled PGD | Transfer FGSM (Student→Teacher) | Transfer FGSM (Teacher→Student) | Transfer FGSM (Teacher→Distilled)
      0 |         72.74 |         99.69 |           78.88 |       72.74 |        99.69 |          78.88 |                       80.66 |                          43.89 |                           71.27
   0.01 |         80.05 |         41.80 |           70.72 |       50.58 |        23.94 |          69.98 |                       80.79 |                          44.08 |                           71.21
   0.05 |         77.59 |         37.26 |           68.82 |        0.68 |         2.21 |          63.35 |                       80.17 |                          45.12 |                           71.09
    0.1 |         76.37 |         35.42 |           66.30 |        0.12 |         0.31 |          59.98 |                       78.70 | 

In [ ]:
# Collect accuracy lists for each model and each attack
fgsm_accs_student = [evaluate_model(student_model, test_loader, eps, attack="fgsm") for eps in epsilons]
fgsm_accs_distilled = [evaluate_model(student_model_distilled, test_loader, eps, attack="fgsm") for eps in epsilons]

pgd_accs_student = [evaluate_model(student_model, test_loader, eps, attack="pgd") for eps in epsilons]
pgd_accs_distilled = [evaluate_model(student_model_distilled, test_loader, eps, attack="pgd") for eps in epsilons]

transfer_accs_t2s = [transfer_attack(teacher_model, student_model, test_loader, eps, fgsm_attack) for eps in epsilons]
transfer_accs_t2d = [transfer_attack(teacher_model, student_model_distilled, test_loader, eps, fgsm_attack) for eps in epsilons]

# Plotting
plt.figure(figsize=(12, 7))
plt.plot(epsilons, fgsm_accs_student, marker='o', label='FGSM (Student)')
plt.plot(epsilons, fgsm_accs_distilled, marker='o', linestyle='--', label='FGSM (Distilled Student)')

plt.plot(epsilons, pgd_accs_student, marker='s', label='PGD (Student)')
plt.plot(epsilons, pgd_accs_distilled, marker='s', linestyle='--', label='PGD (Distilled Student)')

plt.plot(epsilons, transfer_accs_t2s, marker='^', label='Transfer FGSM (Teacher→Student)')
plt.plot(epsilons, transfer_accs_t2d, marker='^', linestyle='--', label='Transfer FGSM (Teacher→Distilled)')

plt.xlabel("Epsilon (ε)")
plt.ylabel("Accuracy (%)")
plt.title("Student Models Under FGSM, PGD, and Transfer Attacks")
plt.legend()
plt.grid(True)
plt.show()